# Dermatologist AI To Detect Skin Cancer

Skin Cancer is the most common cancer in the world. In U.S., there are 5.4 million new cases every year. They come in different types. Some are called Carcinomas and some are called Melanomas. Melanomas are the ones that typically kill people, its called the Black Cancer. 20% of Americans eventually get skin cancer. In most cases benign. Pre-cancer also called Actinic Kertosis, affects 58 miliion Americans and many more in the world. In this country there are 76,000 Melanomas each year and 10,000 deaths. Skin cancers comes in multiple stages from stage zero to stage four. 

According to the American Cancer Society, the 5-year survival rate for stage 4 melanoma is 15–20 percent. This means that an estimated 15–20 percent of people with stage 4 melanoma will be alive 5 years after diagnosis. But the 5-year survival rate for Stage 0, is 98.4%. Therefore the early detection is paramount to not dying from skin cancer.

Dermatologists are extremely highly trained to find melanomas and carcinomas, because they need to make life and death decisions in diagnosing patient's skin. But they are very few in numbers and the early detection is still a problem. If human intelligence is also aided with an highly trained AI deep neural networks, it can revolutionize and save the world, by early detection.


## Goal
Skin cancer is primarily diagnosed visually, beginning with an initial clinical screening and followed potentially by dermoscopic analysis, a biopsy and histopathological examination. Automated classification of skin lesions using images is a challenging task owing to the fine-grained variability in the appearance of skin lesions. Deep convolutional neural networks (CNNs) show potential for general and highly variable tasks across many fine-grained object categories [[ 1 ] ](https://www.nature.com/articles/nature21056.epdf?author_access_token=8oxIcYWf5UNrNpHsUHd2StRgN0jAjWel9jnR3ZoTv0NXpMHRAJy8Qn10ys2O4tuPakXos4UhQAFZ750CsBNMMsISFHIKinKDMKjShCpHIlYPYUHhNzkn6pSnOCt0Ftf6)

The goal of this project is to design an algorithm that can visually diagnose melanoma. In particular, the algorithm will distinguish this malignant skin tumor from two types of benign lesions (nevi and seborrheic keratoses).

## Roadmap

The notebook is broken into below steps:

* Step 0: Setup Google Colab
* Step 1: Imports and CUDA check
* Step 2: Load and Transform Data
* Step 3: Define Model
* Step 4: Final Classifier Layer
* Step 5: Training
* Step 6: Testing
* Step 7: Visualize Sample Test Results


## Set up Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if PIL works
from PIL import Image
Image.open("/content/drive/My Drive/Colab Notebooks/dermatologist_AI/data/train/melanoma/ISIC_0000002.jpg").convert('RGB')

## Imports

In [3]:
import numpy as np
import torch
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [4]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...




> ## Load and Transform Data



In [5]:
# define training and test data directories
data_dir = '/content/drive/My Drive/Colab Notebooks/dermatologist_AI/data/'
train_dir = 'train/'
valid_dir = 'valid/'
test_dir = 'test/'

# classes are folders in each directory with these names
classes = ['melanoma', 'nevus', 'seborrheic_keratosis']

## Transform Data

In [6]:
# Load and transform data using Image Folder
batch_size = 20
num_workers = 0
#Inception-v3 299x299 images as input, so we resize all of them
transform = transforms.Compose([transforms.Resize(320),
                                transforms.CenterCrop(299),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.485, 0.456, 0.406],
                                                     std = [0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir + train_dir,transform=transform )
#train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
valid_data = datasets.ImageFolder(data_dir + valid_dir, transform=transform)
test_data = datasets.ImageFolder(data_dir + test_dir, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)

loaders = {'train' : train_loader,
           'valid' : valid_loader,
           'test' : test_loader}

In [ ]:
class_names = train_data.classes
nb_classes = len(class_names)

print("Number of classes:", nb_classes)
print("\nClass names: \n\n", class_names)

Number of classes: 3

Class names: 

 ['melanoma', 'nevus', 'seborrheic_keratosis']


## Data Visualization

In [ ]:
# Visualize some sample data
inputs, classes = next(iter(train_loader))

for image, label in zip(inputs, classes): 
    image = image.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)     
    fig = plt.figure(figsize=(12,3))
    plt.imshow(image)
    plt.title(class_names[label])

## Model Definition
Here I've used a pretrained model Inception_v3.
Also called GoogleNetv3, a famous ConvNet trained on Imagenet from 2015


In [ ]:
model = models.inception_v3(pretrained = True)
#print(model)

In [8]:
# Freeze training for all conv layers
for param in model.parameters():
  param.requires_grad = False

## Final Classifier Layer

In [9]:
import torch.nn as nn
from collections import OrderedDict

# add last linear layer (n_inputs -> 133 dog's breed classes)
n_output = 3
n_input = model.fc.in_features

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 512)),
                                         ('relu', nn.ReLU()),
                                         ('drop', nn.Dropout(0.5)),
                                         ('fc2', nn.Linear(512, 3)), 
                                         ('output', nn.Softmax(dim=1))]))
model.fc = classifier
#print(model)

if train_on_gpu:
    model = model.cuda()

In [10]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.011)

## Training

In [ ]:
def accuracy(validloader):    
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_size, (data,target) in enumerate(valid_loader):
          if train_on_gpu:
            data, target = data.cuda(), target.cuda()
    
          # forward pass
          output, _ = model(data)
          # calculate teh batch loss
          _, predicted = torch.max(output.data, 1)
          total += target.size(0)
          correct += (predicted == target).sum().item()

        print('Accuracy of the network %d %%' % (100 * correct / total))


accuracy(valid_loader)

In [ ]:
  if model is not None:
    print("in if")
    del model
    torch.cuda.empty_cache()

in if


In [ ]:
n_epochs = 10
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf 
for epoch in range(1, n_epochs+1):
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model.train() # prep model for training
  for batch_size, (data,target) in enumerate(train_loader):    
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
    #clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass
    output,_ = model(data)
    # calculate teh batch loss
    loss = criterion(output, target)
    # backward pass
    loss.backward()
    # optimization step
    optimizer.step()
    # update training loss
    train_loss += loss.item() * data.size(0)

  ###################
  # train the model #
  ###################
  model.eval() # prep model for evaluation
  for batch_size, (data,target) in enumerate(valid_loader):
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()
    
    # forward pass
    output = model(data)
    # calculate teh batch loss
    loss = criterion(output, target)
    #loss = criterion(output, target)
    valid_loss += loss.item()*data.size(0)
  # print training/validation statistics 
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader.sampler)
  valid_loss = valid_loss/len(valid_loader.sampler)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
      epoch+1, 
      train_loss,
      valid_loss
      ))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
    torch.save(model.state_dict(), "./model.pth")
    valid_loss_min = valid_loss


Epoch: 2 	Training Loss: 0.925408 	Validation Loss: 1.029301
Validation loss decreased (inf --> 1.029301).  Saving model ...


KeyboardInterrupt: ignored

In [ ]:
# load the model that got the best validation accuracy
model.load_state_dict(torch.load('./model.pth'))

<All keys matched successfully>

In [ ]:
# track test loss 
# over 3 classes
test_loss = 0.0
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))

model.eval() # eval mode

# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(3):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            class_names[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (class_names[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.044302

Test Accuracy of melanoma:  0% ( 0/20)
Test Accuracy of nevus: 100% (45/45)
Test Accuracy of seborrheic_keratosis:  0% ( 0/19)

Test Accuracy (Overall): 53% (45/84)


In [ ]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title("{} ({})".format(class_names[preds[idx]], class_names[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))